### Callbacks

Before promises, developers used callbacks to handle asynchronous actions. If you requested expenses from a data source, you’d pass a callback function as an argument. After the asynchronous data is returned (or resolved as it is often called) the function would execute the callback. 

The traditional example is a `setTimeout()` function that takes a callback and executes it after certain number of milliseconds. Use `setTimeout()` as a place holder for any action that doesn’t immediately resolve. For example, think about a function called `getUserPreferences()`, which
would fetch data from an API and then pass that data to a callback.

Because Javascript is asynchronous, you can call other functions before and after the call to `getUserPreferences()` and they’d both resolve before `getUserPreferences()` executes the callback:

In [5]:
function getUserPreferences(callback) {
    setTimeout(() => {
        callback({
            theme: 'dusk',
        });
    }, 1000);
}

function log(value) {
    return console.log(value);
}

In [2]:
log('starting');
getUserPreferences((preferences) => {
    return log(preferences.theme.toUpperCase());
});
log('ending?');

starting
ending?
DUSK


**Case Study: Nested callbacks**

The problem with callbacks is that you may have asynchronous functions that call asynchronous functions, that call asynchronous functions, and so on. Eventually you have so many nested callbacks, you find yourself in what's known as “callback hell.”

Here's a function that takes a callback, and calls it after calling its own asynchronous function:

In [3]:
function getMusic(theme, callback) {
    setTimeout(() => {
        if (theme === 'dusk') {
            return callback({ album: 'music for airports' });
        }
        return callback({ album: 'kind of blue' });
    }, 1000);
}

So now `getUserPreferences` takes a callback, which itself passes in another callback to `getMusic`:

In [5]:
getUserPreferences((preferences) => {
    return getMusic(preferences.theme, (music) => {
        log(music.album);
    })
})

music for airports


This function is only nested two deep, and it’s already getting hard to read. Things get complicated fast.

### Promises

A promise is an object that takes asynchronous action and calls one of two methods based on the response. If the asynchronous action is successful, or fulfilled, the promise passes the results to a `then()` method. If the action fails, or is rejected, the promise calls the `catch()` method. Both `then()` and `catch()` take a function as an argument, and that can only take a single response argument.

When you create a promise, you call the `Promise` constructor with two arguments: `resolve()` and `reject()`. `resolve()` is what happens when things go as planned. When `resolve()` is called, the code will execute the function passed to the `then()` method. 

When you define your `getUserPreferences()` function, you’ll return the promise. When you actually call `getUserPreferences()`, you’ll call either the `then()` or the `catch()` method:

In [6]:
function getUserPreferences() {
    const preferences = new Promise((resolve, reject) => {
        resolve({ theme: 'dusk' });
    });
    return preferences;
}

In [8]:
getUserPreferences()
    .then((preferences) => log(preferences.theme))
    .catch((error) => log(`Fail: ${error.type}`));

dusk


Now let's convert `getMusic()` to a promise:

In [1]:
function getMusic(theme) {
    if (theme === 'dusk') {
        return Promise.resolve({ album: 'Music for Airports' });
    }
    return Promise.resolve({ album: 'Kind of Blue' });
}

So now when we call `getUserPreferences()` it returns a promise, and then when it resolves, we call `getMusic()`, which returns a promise, and when that promise resolves we call `log()`:

In [7]:
getUserPreferences()
    .then((preferences) => getMusic(preferences.theme))
    .then((music) => log(music.album))
    .catch((error) => log(`Fail: ${error.type}`))

Music for Airports


**To Do: Promise.all**

### Async and Await